# Dolby.io OAuth Authentication Example

This notebook will walk through how to get started using [OAuth](https://docs.dolby.io/media-processing/docs/authentication) for Dolby.io API key authorization instead of an API key.


## Step 1: Create the Token

Here we want to create our token by taking our API key and secret, and Base64 encoding it to be saved as an environment variable. Note that this will save the Environment variable locally in the environment the code is called, so make sure it is saved somewhere usable by the necessary functions. The token will expire after 12 hours.

In [ ]:
# ! pip install --user requests

import requests
from requests.auth import HTTPBasicAuth

key = input("Input your Dolby.io Media Processing API Key:")
secret = input("Input your Dolby.io Media Processing Secret:")
response = requests.post("https://api.dolby.com/media/oauth2/token",
    auth = HTTPBasicAuth(key, secret),
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data= { "grant_type": "client_credentials" })
response.raise_for_status()
# Note: the token will expire after 12 hours, in which it will need to be regenerated
os.environ['DOLBYIO_API_TOKEN'] = response.json()["access_token"]
print("Token created and saved to environment variable \"DOLBYIO_API_KEY\".")

## Step 2: use the token in an API call

Here, we replace `"x-api-key": api_key,` with `"Authorization": "Bearer <YOUR_TOKEN>"`, with the remainder of the call remaining the same.

In [ ]:
import os
import requests

# Set or replace these values
body = {
  "input" : "dlb://in/example.mp4",
  "output" : "dlb://out/example-enhanced.mp4"
}

url = "https://api.dolby.com/media/enhance"
headers = {
  "Authorization": f"Bearer {os.environ['DOLBYIO_API_TOKEN']}",
  "Content-Type": "application/json",
  "Accept": "application/json"
}

response = requests.post(url, json=body, headers=headers)
response.raise_for_status()
json = response.json()
os.environ['DOLBYIO_JOB_ID'] = json['job_id']
print(f"File uploaded successfully with the job id: {json['job_id']}")

In [ ]:
import os
import requests

url = "https://api.dolby.com/media/enhance"
headers = {
  "Authorization": f"Bearer {os.environ['DOLBYIO_API_TOKEN']}",
  "Content-Type": "application/json",
  "Accept": "application/json"
}

params = {
  "job_id": os.environ["DOLBYIO_JOB_ID"]
}

response = requests.get(url, params=params, headers=headers)
response.raise_for_status()
print(response.json())

In [ ]:
import os
import shutil
import requests

output_path = os.environ["OUTPUT_MEDIA_LOCAL_PATH"]

url = "https://api.dolby.com/media/output"
headers = {
    "Authorization": f"Bearer {os.environ['DOLBYIO_API_TOKEN']}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

args = {
    "url": "dlb://out/example-enhanced.mp4",
}

with requests.get(url, params=args, headers=headers, stream=True) as response:
    response.raise_for_status()
    response.raw.decode_content = True
    print("Downloading from {0} into {1}".format(response.url, output_path))
    with open(output_path, "wb") as output_file:
        shutil.copyfileobj(response.raw, output_file)

## Expanding Further

This is only the beginning of authentication in Python. We can do more to automate the process further for quality of life improvements, such as:
- Storing the token as an automatically expiring cookie if using a web framework, like Flask or Django
- Prompting the token to be recreated upon getting a bad request error on API calls
- Checking the token's validity before it is saved
- Creating a GUI for authentication in a login screen format
- Implementing [Webhooks](https://docs.dolby.io/media-processing/docs/webhooks-and-callbacks) instead of polling for jobs to complete.
- Trying out our [Analyze](https://docs.dolby.io/media-processing/docs/quick-start-to-analyzing-media) or [Diagnose](https://docs.dolby.io/media-processing/docs/quick-start-to-diagnosing-media) API endpoints instead of Enhance for more fine tuned data.
- Adding in real time loading bars and messages.
- Connecting this with a third party cloud storage provider, such as [AWS S3](https://docs.dolby.io/media-processing/docs/aws-s3), instead of local downloads.


### Made by [Dolby.io](https://dolby.io/).

Create your account for free [here](https://dolby.io/signup)!